In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [12]:
def preprocess_stock_data(file_path, output_dir):
    # Load data
    data = pd.read_csv(file_path, parse_dates=['timestamp'], index_col='timestamp')
    
    # Resample data into 10-minute intervals
    volume_resampled = data['volume'].resample('10T').sum()  # Sum volume
    
    # Calculate returns based on open price at the start and close price at the end of the interval
    open_resampled = data['open'].resample('10T').first()
    close_resampled = data['close'].resample('10T').last()
    returns_resampled = (close_resampled - open_resampled) / open_resampled
    
    # Combine into a new DataFrame
    resampled_data = pd.DataFrame({
        'open': open_resampled,
        'close': close_resampled,
        'volume': volume_resampled,
        'returns': returns_resampled
    }).dropna()  # Drop any rows with NaN values which may arise from empty intervals

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Extract the company name from the file path and create the output file name
    company_name = os.path.basename(file_path).replace('.csv', '')
    output_file_path = os.path.join(output_dir, f'Updated_{company_name}.csv')
    
    # Save the resampled data to the new file
    resampled_data.to_csv(output_file_path)
    print(f"Preprocessed data saved to {output_file_path}")

In [13]:
import os

def preprocess_all_data(root_folder, output_root):
    """
    Preprocess all stock data files in the given root folder and outputs them to a new structure under output_root.
    """
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, root_folder)
                output_dir = os.path.join(output_root, relative_path)
                
                preprocess_stock_data(file_path, output_dir)

# Define your original data folder and output folder
root_folder = 'Data'  # Adjust if your root data folder has a different name or path
output_root = 'preprocessed_data'

preprocess_all_data(root_folder, output_root)


Preprocessed data saved to preprocessed_data/./Updated_SPY_Data.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_XRAY_Data_enhanced.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_RHI_Data.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_NCLH_Data_enhanced.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_CMA_Data.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_MHK_Data.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_BWA_Data.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_VFC_Data_enhanced.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_ZION_Data_enhanced.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_CMA_Data_enhanced.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_ETSY_Data.csv
Preprocessed data saved to preprocessed_data/Bottom_20/Updated_BWA_Data_enhanced.csv
Preprocessed data saved to preprocesse

In [16]:
def collect_timestamps(root_folder):
    all_timestamps = []
    
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                data = pd.read_csv(file_path)
                
                # Ensure timestamp is in a consistent format, if necessary
                data['timestamp'] = pd.to_datetime(data['timestamp']).dt.floor('T')  # Floor to minute precision
                file_timestamps = set(data['timestamp'].astype(str))  # Convert to string for easy comparison
                
                all_timestamps.append(file_timestamps)
    
    # Find the intersection of timestamps across all files
    common_timestamps = set.intersection(*all_timestamps)
    
    return common_timestamps


In [18]:
def filter_datasets_to_common_timestamps(root_folder, common_timestamps):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.csv'):
                input_path = os.path.join(root, file)
                data = pd.read_csv(input_path)
                
                data['timestamp'] = pd.to_datetime(data['timestamp']).dt.floor('T')
                data['timestamp'] = data['timestamp'].astype(str)
                
                data_filtered = data[data['timestamp'].isin(common_timestamps)]
                
                # Save the filtered dataset back to its original location
                data_filtered.to_csv(input_path, index=False)
                print(f"Filtered dataset updated at {input_path}")


In [19]:
root_folder = 'preprocessed_data'

# Collect common timestamps across all datasets
common_timestamps = collect_timestamps(root_folder)

# Filter each dataset to only include common timestamps and update them in-place
filter_datasets_to_common_timestamps(root_folder, common_timestamps)

Filtered dataset updated at preprocessed_data/Updated_SPY_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_XRAY_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_FRT_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_MKTX_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_IVZ_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_PNW_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_VFC_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_DOC_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_BWA_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_RHI_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_ETSY_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_MHK_Data.csv
Filtered dataset updated at preprocessed_data/Bottom_20/Updated_CMA_Data.csv
Filter

In [23]:
def add_mag7_spy_to_bottom20(mag7_folder, bottom20_folder, spy_file):
    # Load SPY data
    spy_data = pd.read_csv(spy_file, index_col='timestamp', parse_dates=['timestamp'])
    
    # Load Mag_7 data into a dictionary
    mag7_data = {}
    for filename in os.listdir(mag7_folder):
        if filename.startswith("Updated_") and filename.endswith("_Data.csv"):
            filepath = os.path.join(mag7_folder, filename)
            # Extract company name considering the 'Updated_' prefix and '_Data.csv' suffix
            company_name = filename.replace("Updated_", "").replace("_Data.csv", "")
            mag7_data[company_name] = pd.read_csv(filepath, index_col='timestamp', parse_dates=['timestamp'])

    # Iterate through Bottom_20 and add Mag_7 and SPY data
    for bottom20_file in os.listdir(bottom20_folder):
        if bottom20_file.startswith("Updated_") and bottom20_file.endswith("_Data.csv"):
            bottom20_filepath = os.path.join(bottom20_folder, bottom20_file)
            bottom20_data = pd.read_csv(bottom20_filepath, index_col='timestamp', parse_dates=['timestamp'])

            # Add Mag_7 volume and returns
            for company_name, df in mag7_data.items():
                bottom20_data[f'{company_name} Volume'] = df['volume'].reindex(bottom20_data.index).fillna(0)
                bottom20_data[f'{company_name} Returns'] = df['returns'].reindex(bottom20_data.index).fillna(0)

            # Add SPY volume and returns
            bottom20_data['SPY Volume'] = spy_data['volume'].reindex(bottom20_data.index).fillna(0)
            bottom20_data['SPY Returns'] = spy_data['returns'].reindex(bottom20_data.index).fillna(0)

            # Save the updated Bottom_20 data
            bottom20_data.to_csv(bottom20_filepath)
            print(f"Updated {bottom20_file} with Mag_7 and SPY data.")


In [24]:
mag7_folder = 'preprocessed_data/Mag_7'
bottom20_folder = 'preprocessed_data/Bottom_20'
spy_file = 'preprocessed_data/Updated_SPY_Data.csv'  # Adjust if your filename differs

add_mag7_spy_to_bottom20(mag7_folder, bottom20_folder, spy_file)


Updated Updated_XRAY_Data.csv with Mag_7 and SPY data.
Updated Updated_FRT_Data.csv with Mag_7 and SPY data.
Updated Updated_MKTX_Data.csv with Mag_7 and SPY data.
Updated Updated_IVZ_Data.csv with Mag_7 and SPY data.
Updated Updated_PNW_Data.csv with Mag_7 and SPY data.
Updated Updated_VFC_Data.csv with Mag_7 and SPY data.
Updated Updated_DOC_Data.csv with Mag_7 and SPY data.
Updated Updated_BWA_Data.csv with Mag_7 and SPY data.
Updated Updated_RHI_Data.csv with Mag_7 and SPY data.
Updated Updated_ETSY_Data.csv with Mag_7 and SPY data.
Updated Updated_MHK_Data.csv with Mag_7 and SPY data.
Updated Updated_CMA_Data.csv with Mag_7 and SPY data.
Updated Updated_GNRC_Data.csv with Mag_7 and SPY data.
Updated Updated_NCLH_Data.csv with Mag_7 and SPY data.
Updated Updated_HAS_Data.csv with Mag_7 and SPY data.
Updated Updated_WHR_Data.csv with Mag_7 and SPY data.
Updated Updated_ZION_Data.csv with Mag_7 and SPY data.
Updated Updated_PARA_Data.csv with Mag_7 and SPY data.
Updated Updated_FMC_D

In [26]:
def generate_train_test_slices(file_path, start_date, end_date, train_minutes, test_minutes):
    # Constants
    interval_length = 10  # Each observation covers a 10-minute interval
    mag7_companies = ['AAPL', 'AMZN', 'MSFT', 'TSLA', 'META', 'NVDA', 'GOOG']
    
    # Load data
    data = pd.read_csv(file_path, parse_dates=['timestamp'], index_col='timestamp')
    
    # Filter data by the specified date range
    data = data[(data.index.date >= pd.to_datetime(start_date).date()) & 
                (data.index.date <= pd.to_datetime(end_date).date())]
    
    # Identify Mag_7 and SPY column names
    mag7_columns = [col for col in data.columns if any(company in col for company in mag7_companies)]
    spy_columns = [col for col in data.columns if 'SPY' in col]
    
    # Initialize lists for train and test slices
    mag7_train_slices = []
    spy_train_slices = []
    test_slices = []
    
    # Calculate the number of intervals for training and testing
    train_intervals = train_minutes // interval_length
    test_intervals = test_minutes // interval_length
    
    start_idx = 0
    total_intervals = len(data)
    
    # Loop to slice the data for training and testing intervals
    while start_idx + train_intervals + test_intervals <= total_intervals:
        # Training and testing indices
        end_train_idx = start_idx + train_intervals
        start_test_idx = end_train_idx
        end_test_idx = start_test_idx + test_intervals
        
        # Append train slices
        mag7_train_slices.append(data.iloc[start_idx:end_train_idx][mag7_columns])
        spy_train_slices.append(data.iloc[start_idx:end_train_idx][spy_columns])
        
        # Append test slice (only once per cycle since it's the same for both train sets)
        test_slices.append(data.iloc[start_test_idx:end_test_idx])
        
        # Update the start index for the next slicing cycle
        start_idx = end_test_idx
    
    return mag7_train_slices, spy_train_slices, test_slices


In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

def train_and_evaluate(train_slice_mag7, train_slice_spy, test_slice):
    # Prepare target and features for training with Mag7
    y_train = train_slice_mag7['returns']
    X_train_mag7 = train_slice_mag7.drop(columns=['returns'])

    # Prepare features for training with SPY
    X_train_spy = train_slice_spy.drop(columns=['returns'])

    # Prepare target and features for testing
    y_test = test_slice['returns']
    X_test = test_slice.drop(columns=['returns'])

    # Train model on Mag7 data
    model_mag7 = LinearRegression()
    model_mag7.fit(X_train_mag7, y_train)
    predictions_mag7 = model_mag7.predict(X_test)

    # Train model on SPY data
    model_spy = LinearRegression()
    model_spy.fit(X_train_spy, y_train)
    predictions_spy = model_spy.predict(X_test)

    # Calculate RMSE for predictions
    rmse_mag7 = sqrt(mean_squared_error(y_test, predictions_mag7))
    rmse_spy = sqrt(mean_squared_error(y_test, predictions_spy))

    return rmse_mag7, rmse_spy

In [40]:

file_path = 'preprocessed_data/Bottom_20/Updated_MKTX_Data.csv'
start_date = '2023-09-01'
end_date = '2023-09-02'
train_minutes = 180
test_minutes = 30


mag7_train_slices, spy_train_slices, test_slices = generate_train_test_slices(
    file_path, start_date, end_date, train_minutes, test_minutes
)

# Evaluate models and collect RMSE scores
rmse_scores_mag7 = []
rmse_scores_spy = []

for i in range(len(test_slices)):
    rmse_mag7, rmse_spy = train_and_evaluate(mag7_train_slices[i], spy_train_slices[i], test_slices[i])
    rmse_scores_mag7.append(rmse_mag7)
    rmse_scores_spy.append(rmse_spy)
    print(f"Interval {i+1}: RMSE Mag7: {rmse_mag7}, RMSE SPY: {rmse_spy}")

# Output aggregated RMSE results
print("Aggregated RMSE scores for Mag7 training:", np.mean(rmse_scores_mag7))
print("Aggregated RMSE scores for SPY training:", np.mean(rmse_scores_spy))

KeyError: 'returns'

In [33]:
import pandas as pd

# Load the uploaded file
file_path = 'preprocessed_data/Bottom_20/Updated_MKTX_Data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(data.head())


             timestamp    open    close  volume   returns  Updated Volume  \
0  2023-09-01 13:30:00  243.31  242.020    5596 -0.005302       3174465.0   
1  2023-09-01 13:40:00  241.65  241.685    1763  0.000145       1894580.0   
2  2023-09-01 13:50:00  241.37  240.970    3062 -0.001657       1320054.0   
3  2023-09-01 14:00:00  240.92  241.570    4360  0.002698       1762548.0   
4  2023-09-01 14:10:00  242.26  241.475    2439 -0.003240       1400384.0   

   Updated Returns  SPY Volume  SPY Returns  GOOG Volume  ...  AMZN Volume  \
0        -0.005356   3482566.0    -0.002196    1654453.0  ...    3763564.0   
1         0.001114   2197647.0    -0.001482    1063642.0  ...    1862218.0   
2         0.001431   1699129.0     0.001851     689390.0  ...    1405575.0   
3         0.001626   2582852.0     0.000420     595974.0  ...    1312147.0   
4         0.000634   1984999.0     0.000928     472404.0  ...    1351848.0   

   AMZN Returns  META Volume  META Returns  TSLA Volume  TSLA Return